# 0/1 Knapsack problem

In [1]:
function ks(i, w, wt, vl)
    if i == 0
        0
    elseif w == 0
        0
    elseif wt[i] > w
        ks(i - 1, w, wt, vl)
    else
        r1 = ks(i - 1, w - wt[i], wt, vl)
        r2 = ks(i - 1, w, wt, vl)
        max(vl[i] + r1, r2)
    end
end

ks(w, wt, vl) = ks(length(wt), w, wt, vl)

ks (generic function with 2 methods)

In [2]:
using Test

In [3]:
@test ks(50, (10, 20, 30), (100, 200, 300)) == 500

Test Passed

## Специализация по `w` и `wt`

In [4]:
using MacroTools: prettify

In [5]:
function ks_gen_impl1(i, w, wt)
    if i == 0
        0
    elseif w == 0
        0
    elseif wt[i] > w
        ks_gen_impl1(i - 1, w, wt)
    else
        r1 = ks_gen_impl1(i - 1, w - wt[i], wt)
        r2 = ks_gen_impl1(i - 1, w, wt)
        :(max(vl[$i] + $r1, $r2))
    end
end

ks_gen_impl1(w, wt) = ks_gen_impl1(length(wt), w, wt)

ks_gen_impl1 (generic function with 2 methods)

In [6]:
ks_gen_impl1(50, (10, 20, 30)) |> prettify

:(max(vl[3] + max(vl[2] + 0, max(vl[1] + 0, 0)), max(vl[2] + max(vl[1] + 0, 0), max(vl[1] + 0, 0))))

## Мемоизация

In [7]:
function ass!(es, d, s::Symbol, c::Int64)
    d[s] = c
end

function ass!(es, d, s::Symbol, u::Symbol)
    d[s] = u
end

function ass!(es, d, s::Symbol, e)
    haskey(d, s) && return
    d[s] = s
    push!(es, :($s = $e))
end

ass! (generic function with 3 methods)

In [8]:
function ks_gen_impl2!(es, d, i, w, wt)
    r = Symbol("r_", i, "_", w)

    if haskey(d, r)
        return d[r]
    end

    if i == 0
        ass!(es, d, r, 0)
    elseif w == 0
        ass!(es, d, r, 0)
    elseif wt[i] > w
        ass!(es, d, r, ks_gen_impl2!(es, d, i - 1, w, wt))
    else
        r1 = ks_gen_impl2!(es, d, i - 1, w - wt[i], wt)
        r2 = ks_gen_impl2!(es, d, i - 1, w, wt)
        ass!(es, d, r, :(max(vl[$i] + $r1, $r2)))
    end
    return d[r]
end

ks_gen_impl2! (generic function with 1 method)

In [9]:
function ks_gen_impl2(w, wt)
    es = Expr[]
    d = Dict{Symbol,Any}()
    r = ks_gen_impl2!(es, d, length(wt), w, wt)

    quote
        $(es...)
        return $r
    end
end

ks_gen_impl2 (generic function with 1 method)

In [10]:
ks_gen_impl2(50, (10, 20, 30)) |> prettify

quote
    r_1_20 = max(vl[1] + 0, 0)
    r_2_20 = max(vl[2] + 0, r_1_20)
    r_1_30 = max(vl[1] + 0, 0)
    r_1_50 = max(vl[1] + 0, 0)
    r_2_50 = max(vl[2] + r_1_30, r_1_50)
    r_3_50 = max(vl[3] + r_2_20, r_2_50)
    return r_3_50
end

In [11]:
add_s(x, y::Int64) =
    if y == 0
        x
    else
        :($x + $y)
    end

add_s(x, y) = :($x + $y)

add_s (generic function with 2 methods)

In [12]:
max_s(x, y::Int64) =
    if y == 0
        x
    else
        :(max($x, $y))
    end

max_s(x, y) = :(max($x, $y))

max_s (generic function with 2 methods)

In [13]:
function ks_gen_impl3!(es, d, i, w, wt)
    r = Symbol("r_", i, "_", w)

    if haskey(d, r)
        return d[r]
    end

    if i == 0
        ass!(es, d, r, 0)
    elseif w == 0
        ass!(es, d, r, 0)
    elseif wt[i] > w
        ass!(es, d, r, ks_gen_impl3!(es, d, i - 1, w, wt))
    else
        r1 = ks_gen_impl3!(es, d, i - 1, w - wt[i], wt)
        r2 = ks_gen_impl3!(es, d, i - 1, w, wt)
        vl_i = Symbol("vl_", i)
        ass!(es, d, vl_i, :(vl[$i]))
        ass!(es, d, r, max_s(add_s(vl_i, r1), r2))
    end
    return d[r]
end

ks_gen_impl3! (generic function with 1 method)

In [14]:
function ks_gen_impl3(w, wt)
    es = Expr[]
    d = Dict{Symbol,Any}()
    r = ks_gen_impl3!(es, d, length(wt), w, wt)

    quote
        $(es...)
        return $r
    end
end

ks_gen_impl3 (generic function with 1 method)

In [15]:
ks_gen_impl3(50, (10, 20, 30)) |> prettify

quote
    vl_1 = vl[1]
    vl_2 = vl[2]
    r_2_20 = max(vl_2, vl_1)
    r_2_50 = max(vl_2 + vl_1, vl_1)
    vl_3 = vl[3]
    r_3_50 = max(vl_3 + r_2_20, r_2_50)
    return r_3_50
end

In [16]:
ks_gen_impl3(60, (70, 10, 50, 20, 30, 40, 60)) |> prettify

quote
    vl_2 = vl[2]
    vl_4 = vl[4]
    r_4_20 = max(vl_4, vl_2)
    r_4_30 = max(vl_4 + vl_2, vl_2)
    vl_3 = vl[3]
    r_3_60 = max(vl_3 + vl_2, vl_2)
    r_4_60 = max(vl_4 + vl_2, r_3_60)
    vl_5 = vl[5]
    r_5_60 = max(vl_5 + r_4_30, r_4_60)
    vl_6 = vl[6]
    r_6_60 = max(vl_6 + r_4_20, r_5_60)
    vl_7 = vl[7]
    r_7_60 = max(vl_7, r_6_60)
    return r_7_60
end

In [17]:
@generated function ks_gen3(::Val{w}, ::Val{wl}, vl) where {w,wl}
    ks_gen_impl3(w, wl)
end

ks_gen3 (generic function with 1 method)

In [18]:
@test ks_gen3(Val(50), Val((10, 20, 30)), (100, 200, 300)) == 500

Test Passed

## Дополнительная оптимизация

In [19]:
using MacroTools: postwalk, @capture

In [20]:
function opt_step(e)
    if @capture(e, x_ + 0)
        x
    elseif @capture(e, max(x_, 0))
        x
    elseif @capture(e, max(x_ + y1_, y2_)) && y1 == y2
        :($x + $y1)
    else
        e
    end
end

opt_expr(e) = postwalk(opt_step, e)

opt_expr (generic function with 1 method)

In [21]:
@test opt_expr(:(a + c)) == :(a + c)
@test opt_expr(:(a + 0)) == :a

@test opt_expr(:(max(a, c))) == :(max(a, c))
@test opt_expr(:(max(a, 0))) == :a

@test opt_expr(:(max(a + b, c))) == :(max(a + b, c))
@test opt_expr(:(max(a + (b + c), b + c))) == :(a + (b + c))

@test opt_expr(:(max(a + b, b + 0))) == :(a + b)

Test Passed

In [22]:
function ks_gen_impl4!(es, d, i, w, wt)
    r = Symbol("r_", i, "_", w)

    if haskey(d, r)
        return d[r]
    end

    if i == 0
        ass!(es, d, r, 0)
    elseif w == 0
        ass!(es, d, r, 0)
    elseif wt[i] > w
        ass!(es, d, r, ks_gen_impl4!(es, d, i - 1, w, wt))
    else
        r1 = ks_gen_impl4!(es, d, i - 1, w - wt[i], wt)
        r2 = ks_gen_impl4!(es, d, i - 1, w, wt)
        vl_i = Symbol("vl_", i)
        ass!(es, d, vl_i, :(vl[$i]))
        ass!(es, d, r, opt_expr(
            :(max($vl_i + $r1, $r2))
        ))
    end
    return d[r]
end

ks_gen_impl4! (generic function with 1 method)

In [23]:
function ks_gen_impl4(w, wt)
    es = Expr[]
    d = Dict{Symbol,Any}()
    r = ks_gen_impl4!(es, d, length(wt), w, wt)

    quote
        $(es...)
        return $r
    end
end

ks_gen_impl4 (generic function with 1 method)

In [24]:
ks_gen_impl4(50, (10, 20, 30)) |> prettify

quote
    vl_1 = vl[1]
    vl_2 = vl[2]
    r_2_20 = max(vl_2, vl_1)
    r_2_50 = vl_2 + vl_1
    vl_3 = vl[3]
    r_3_50 = max(vl_3 + r_2_20, r_2_50)
    return r_3_50
end

In [25]:
ks_gen_impl4(60, (70, 10, 50, 20, 30, 40, 60)) |> prettify

quote
    vl_2 = vl[2]
    vl_4 = vl[4]
    r_4_20 = max(vl_4, vl_2)
    r_4_30 = vl_4 + vl_2
    vl_3 = vl[3]
    r_3_60 = vl_3 + vl_2
    r_4_60 = max(vl_4 + vl_2, r_3_60)
    vl_5 = vl[5]
    r_5_60 = max(vl_5 + r_4_30, r_4_60)
    vl_6 = vl[6]
    r_6_60 = max(vl_6 + r_4_20, r_5_60)
    vl_7 = vl[7]
    r_7_60 = max(vl_7, r_6_60)
    return r_7_60
end

In [26]:
@generated function ks_gen4(::Val{w}, ::Val{wl}, vl) where {w,wl}
    ks_gen_impl4(w, wl)
end

ks_gen4 (generic function with 1 method)

In [27]:
@test ks_gen4(Val(50), Val((10, 20, 30)), (100, 200, 300)) == 500

Test Passed